# 1 Initialize

In [1]:
visible_devices = 1
print("Visible devices:", visible_devices)
import datetime
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing os...")
import os
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing re...")
import re
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing timer from timeit...")
from timeit import default_timer as timer
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Setting env variables for tf import (only device", visible_devices, "will be available)...")
os.environ["CUDA_VISIBLE_DEVICES"] = str(visible_devices)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2'
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing numpy...")
import numpy as np
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing matplotlib...")
from matplotlib import pyplot as plt
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing sys...")
import sys
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing h5py...")
import h5py
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing importlib.util...")
import importlib.util
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing json...")
import json
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing pandas...")
import pandas as pd
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing random...")
import random
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing scipy utils...")
from scipy.stats import norm, chi2, kstwo, kstwobign, ks_2samp
from scipy.special import kolmogorov
from scipy.optimize import minimize, curve_fit, root, bisect
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing subprocess...")
import subprocess
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tensorflow...")
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tensorflow_probability...")
import tensorflow_probability as tfp
tfd = tfp.distributions
print("Tensorflow probability version:", tfp.__version__)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing timeit...")
from timeit import default_timer as timer
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tqdm...")
from tqdm import tqdm
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing typing...")
from typing import List, Tuple, Dict, Callable, Union, Optional, Any, Type
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Setting tf configs...")
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing GMetrics module...")

sys.path.insert(0,'../utils_func/')
import MixtureDistributions # type: ignore

sys.path.insert(0,'../../')
import GenerativeModelsMetrics as GMetrics # type: ignore


paper_fig_dir = "../../../NormalizingFlows/papers/NFHD/figures/"

def get_gpu_info():
    try:
        gpu_info = subprocess.check_output(["nvidia-smi", "--query-gpu=gpu_name", "--format=csv,noheader"]).decode('utf-8')
        return gpu_info.strip().split('\n')
    except Exception as e:
        print(e)
        return None
gpu_models = get_gpu_info()
if gpu_models:
    training_device = gpu_models[eval(os.environ["CUDA_VISIBLE_DEVICES"])]
    print("Successfully loaded GPU model: {}".format(training_device))
else:
    training_device = 'undetermined'
    print("Failed to load GPU model. Defaulting to 'undetermined'.")
    
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "All done.")

Visible devices: 1
2023-07-25 16:59:04.392697: Importing os...
2023-07-25 16:59:04.392782: Importing re...
2023-07-25 16:59:04.392832: Importing timer from timeit...
2023-07-25 16:59:04.392928: Setting env variables for tf import (only device 1 will be available)...
2023-07-25 16:59:04.393029: Importing numpy...
2023-07-25 16:59:04.393080: Importing matplotlib...
2023-07-25 16:59:04.393133: Importing sys...
2023-07-25 16:59:04.393184: Importing h5py...
2023-07-25 16:59:04.413013: Importing importlib.util...
2023-07-25 16:59:04.413116: Importing json...
2023-07-25 16:59:04.413168: Importing pandas...
2023-07-25 16:59:04.640155: Importing random...
2023-07-25 16:59:04.640307: Importing scipy utils...
2023-07-25 16:59:04.905335: Importing subprocess...
2023-07-25 16:59:04.905489: Importing tensorflow...
Tensorflow version: 2.13.0
2023-07-25 16:59:06.473535: Importing tensorflow_probability...
Tensorflow probability version: 0.20.1
2023-07-25 16:59:06.823714: Importing timeit...
2023-07-25

# 2 Usage

## 2.1 use_tf = True, methof = "paired"

### Define distribution

In [2]:
ncomp = 3
ndims = 100
seed = 0
#loc = np.array(np.zeros(ndims,dtype=np.float64),dtype=np.float64)
#scale = np.array(np.ones(ndims,dtype=np.float64),dtype=np.float64)
#dist = tfp.distributions.Normal(loc=loc, scale=scale)
dist = MixtureDistributions.MixtureGaussian(ncomp,ndims,seed)
print(dist.dtype)

<dtype: 'float64'>


### Run test with TensorFlow (small sample with numerical distribution)

In [3]:
dist_1 = tf.cast(dist.sample(1e6),tf.float64)
dist_2 = tf.cast(dist.sample(1e6),tf.float64)

In [4]:
TwoSampleTestInputs_tf = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                      dist_2_input = dist_2,
                                                      niter = 100,
                                                      batch_size = 10000,
                                                      dtype_input = tf.float64,
                                                      seed_input = 0,
                                                      use_tf = True,
                                                      verbose = True)
print("nsamples",TwoSampleTestInputs_tf.nsamples)
print("batch_size",TwoSampleTestInputs_tf.batch_size)
print("niter",TwoSampleTestInputs_tf.niter)
print("niter * batch_size",TwoSampleTestInputs_tf.niter*TwoSampleTestInputs_tf.batch_size)
print("small_sample",TwoSampleTestInputs_tf.small_sample)

Parsing input distribution...


Input distribution is a numeric numpy array or tf.Tensor.
Parsing input distribution...
Input distribution is a numeric numpy array or tf.Tensor.
nsamples 1000000
batch_size 10000
niter 100
niter * batch_size 1000000
small_sample True


In [5]:
KSTest_tf = GMetrics.KSTest(TwoSampleTestInputs_tf,
                            method = "paired",
                            progress_bar = True,
                            verbose = True)

progress_bar is disabled when using tensorflow mode.


In [6]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = True
KSTest_tf.Test_tf(max_vectorize = 1e4)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 10000


Running compute_paired
nchunks = 1
Iterating from 0 to 100 out of 100 .
KS tests calculation completed in 8.730286253034137 seconds.


In [7]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = True
KSTest_tf.Test_np()

Starting two-sample test calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 10000
Two-sample test calculation completed in 40.38134188501863 seconds.


In [12]:
tf.stack([KSTest_tf.Results[-2].result_value,KSTest_tf.Results[-1].result_value])

<tf.Tensor: shape=(100, 2), dtype=float64, numpy=
array([[0.01433101, 0.33765077],
       [0.014228  , 0.34656616],
       [0.01508901, 0.25457329],
       [0.01424101, 0.3270561 ],
       [0.01792401, 0.14157285],
       [0.01104401, 0.59287249],
       [0.01152601, 0.54802747],
       [0.011043  , 0.59761833],
       [0.01284501, 0.43528481],
       [0.010388  , 0.65359116],
       [0.010766  , 0.61881477],
       [0.01257201, 0.45458124],
       [0.01294201, 0.42746784],
       [0.01240001, 0.47473742],
       [0.010464  , 0.64546808],
       [0.01338401, 0.40018952],
       [0.011641  , 0.54830927],
       [0.01341201, 0.38922143],
       [0.01139101, 0.56444236],
       [0.009944  , 0.69133912],
       [0.01101   , 0.60333709],
       [0.013221  , 0.40810443],
       [0.01241001, 0.47795313],
       [0.010568  , 0.63554628],
       [0.01041601, 0.64305147],
       [0.01173101, 0.53276147],
       [0.01212001, 0.49593812],
       [0.012445  , 0.47697057],
       [0.01205201, 0.5004

In [13]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = True
KSTest_tf.Test_tf(max_vectorize = 100)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 10000


Running compute_paired
nchunks = 100
Iterating from 0 to 1 out of 100 .
Iterating from 1 to 2 out of 100 .
Iterating from 2 to 3 out of 100 .
Iterating from 3 to 4 out of 100 .
Iterating from 4 to 5 out of 100 .
Iterating from 5 to 6 out of 100 .
Iterating from 6 to 7 out of 100 .
Iterating from 7 to 8 out of 100 .
Iterating from 8 to 9 out of 100 .
Iterating from 9 to 10 out of 100 .
Iterating from 10 to 11 out of 100 .
Iterating from 11 to 12 out of 100 .
Iterating from 12 to 13 out of 100 .
Iterating from 13 to 14 out of 100 .
Iterating from 14 to 15 out of 100 .
Iterating from 15 to 16 out of 100 .
Iterating from 16 to 17 out of 100 .
Iterating from 17 to 18 out of 100 .
Iterating from 18 to 19 out of 100 .
Iterating from 19 to 20 out of 100 .
Iterating from 20 to 21 out of 100 .
Iterating from 21 to 22 out of 100 .
Iterating from 22 to 23 out of 100 .
Iterating from 23 to 24 out of 100 .
Iterating from 24 to 25 out of 100 .
Iterating from 25 to 26 out of 100 .
Iterating from 26 to

In [14]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(100, 2), dtype=float64, numpy=
array([[0.012852  , 0.46676496],
       [0.012868  , 0.42679842],
       [0.011338  , 0.57115661],
       [0.01193   , 0.51534346],
       [0.016606  , 0.19622785],
       [0.01055601, 0.63363355],
       [0.012441  , 0.47736403],
       [0.013125  , 0.40908823],
       [0.010643  , 0.63246437],
       [0.01302301, 0.42205598],
       [0.01714001, 0.18736855],
       [0.010546  , 0.65097332],
       [0.01338   , 0.39413593],
       [0.011401  , 0.56306905],
       [0.01137201, 0.5725666 ],
       [0.01323401, 0.39977212],
       [0.01193801, 0.51779503],
       [0.01343801, 0.38207958],
       [0.01111501, 0.58189999],
       [0.010973  , 0.61218139],
       [0.01327401, 0.41969436],
       [0.009996  , 0.68907312],
       [0.01177701, 0.54049787],
       [0.011076  , 0.5968196 ],
       [0.011128  , 0.58501515],
       [0.010371  , 0.66319099],
       [0.01159201, 0.5367786 ],
       [0.01286801, 0.44658015],
       [0.013176  , 0.4113

In [15]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = False
KSTest_tf.Test_tf(max_vectorize = 1e4)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 10000


Running compute_paired
nchunks = 1
Iterating from 0 to 100 out of 100 .
KS tests calculation completed in 6.464005699031986 seconds.


In [16]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(100, 2), dtype=float64, numpy=
array([[0.01433101, 0.33765077],
       [0.014228  , 0.34656616],
       [0.01508901, 0.25457329],
       [0.01424101, 0.3270561 ],
       [0.01792401, 0.14157285],
       [0.01104401, 0.59287249],
       [0.01152601, 0.54802747],
       [0.011043  , 0.59761833],
       [0.01284501, 0.43528481],
       [0.010388  , 0.65359116],
       [0.010766  , 0.61881477],
       [0.01257201, 0.45458124],
       [0.01294201, 0.42746784],
       [0.01240001, 0.47473742],
       [0.010464  , 0.64546808],
       [0.01338401, 0.40018952],
       [0.011641  , 0.54830927],
       [0.01341201, 0.38922143],
       [0.01139101, 0.56444236],
       [0.009944  , 0.69133912],
       [0.01101   , 0.60333709],
       [0.013221  , 0.40810443],
       [0.01241001, 0.47795313],
       [0.010568  , 0.63554628],
       [0.01041601, 0.64305147],
       [0.01173101, 0.53276147],
       [0.01212001, 0.49593812],
       [0.012445  , 0.47697057],
       [0.01205201, 0.5004

In [17]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = False
KSTest_tf.Test_tf(max_vectorize = 100)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 10000


Running compute_paired
nchunks = 100
Iterating from 0 to 1 out of 100 .
Iterating from 1 to 2 out of 100 .
Iterating from 2 to 3 out of 100 .
Iterating from 3 to 4 out of 100 .
Iterating from 4 to 5 out of 100 .
Iterating from 5 to 6 out of 100 .
Iterating from 6 to 7 out of 100 .
Iterating from 7 to 8 out of 100 .
Iterating from 8 to 9 out of 100 .
Iterating from 9 to 10 out of 100 .
Iterating from 10 to 11 out of 100 .
Iterating from 11 to 12 out of 100 .
Iterating from 12 to 13 out of 100 .
Iterating from 13 to 14 out of 100 .
Iterating from 14 to 15 out of 100 .
Iterating from 15 to 16 out of 100 .
Iterating from 16 to 17 out of 100 .
Iterating from 17 to 18 out of 100 .
Iterating from 18 to 19 out of 100 .
Iterating from 19 to 20 out of 100 .
Iterating from 20 to 21 out of 100 .
Iterating from 21 to 22 out of 100 .
Iterating from 22 to 23 out of 100 .
Iterating from 23 to 24 out of 100 .
Iterating from 24 to 25 out of 100 .
Iterating from 25 to 26 out of 100 .
Iterating from 26 to

In [18]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(100, 2), dtype=float64, numpy=
array([[0.012852  , 0.46676496],
       [0.012868  , 0.42679842],
       [0.011338  , 0.57115661],
       [0.01193   , 0.51534346],
       [0.016606  , 0.19622785],
       [0.01055601, 0.63363355],
       [0.012441  , 0.47736403],
       [0.013125  , 0.40908823],
       [0.010643  , 0.63246437],
       [0.01302301, 0.42205598],
       [0.01714001, 0.18736855],
       [0.010546  , 0.65097332],
       [0.01338   , 0.39413593],
       [0.011401  , 0.56306905],
       [0.01137201, 0.5725666 ],
       [0.01323401, 0.39977212],
       [0.01193801, 0.51779503],
       [0.01343801, 0.38207958],
       [0.01111501, 0.58189999],
       [0.010973  , 0.61218139],
       [0.01327401, 0.41969436],
       [0.009996  , 0.68907312],
       [0.01177701, 0.54049787],
       [0.011076  , 0.5968196 ],
       [0.011128  , 0.58501515],
       [0.010371  , 0.66319099],
       [0.01159201, 0.5367786 ],
       [0.01286801, 0.44658015],
       [0.013176  , 0.4113

In [25]:
KSTest_tf.Results.results_df

,timestamp,test_name,result_value,is_symb_1,is_symb_2,ndims,niter,batch_size,dtype,small_sample,method,niter_used,batch_size_used,computing_time,backend
0,2023-07-25T14:45:47.285512,KS Test_tf,"((tf.Tensor(0.015589006170630455, shape=(), dt...",True,True,100,100,10000,<dtype: 'float64'>,True,paired,100,10000,7.259382,tensorflow


### Run test with TensorFlow (small sample with symbolic distribution)

In [20]:
dist_1 = dist
dist_2 = dist

In [21]:
TwoSampleTestInputs_tf = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                      dist_2_input = dist_2,
                                                      niter = 100,
                                                      batch_size = 10000,
                                                      dtype_input = tf.float64,
                                                      seed_input = 0,
                                                      use_tf = True,
                                                      verbose = True)
print("nsamples",TwoSampleTestInputs_tf.nsamples)
print("batch_size",TwoSampleTestInputs_tf.batch_size)
print("niter",TwoSampleTestInputs_tf.niter)
print("niter * batch_size",TwoSampleTestInputs_tf.niter*TwoSampleTestInputs_tf.batch_size)
print("small_sample",TwoSampleTestInputs_tf.small_sample)

Parsing input distribution...


Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
nsamples 1000000
batch_size 10000
niter 100
niter * batch_size 1000000
small_sample True


In [22]:
KSTest_tf = GMetrics.KSTest(TwoSampleTestInputs_tf,
                            method = "paired",
                            progress_bar = True,
                            verbose = True)

progress_bar is disabled when using tensorflow mode.


In [23]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = True
KSTest_tf.Test_tf(max_vectorize = 1e4)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 10000
Running compute_paired
nchunks = 1
Iterating from 0 to 100 out of 100 .
KS tests calculation completed in 7.259381621028297 seconds.


In [24]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(100, 2), dtype=float64, numpy=
array([[0.01558901, 0.24454996],
       [0.01039401, 0.64716021],
       [0.01275501, 0.43812501],
       [0.01597901, 0.23338938],
       [0.01257301, 0.45284032],
       [0.01135001, 0.56305815],
       [0.01528001, 0.28003084],
       [0.01176901, 0.53658537],
       [0.011809  , 0.51606922],
       [0.02251601, 0.07712792],
       [0.01685501, 0.19528654],
       [0.010814  , 0.60792263],
       [0.01130801, 0.56469508],
       [0.01223301, 0.48102618],
       [0.015214  , 0.25050486],
       [0.011377  , 0.5646755 ],
       [0.012349  , 0.47441067],
       [0.01206401, 0.49197422],
       [0.01836201, 0.11158024],
       [0.01060801, 0.63319208],
       [0.01129   , 0.57832467],
       [0.01072201, 0.63431611],
       [0.01615701, 0.22392683],
       [0.011728  , 0.52509555],
       [0.01312601, 0.41071273],
       [0.016201  , 0.22843652],
       [0.01078101, 0.62167935],
       [0.01388001, 0.34699128],
       [0.011665  , 0.5368

In [20]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = True
KSTest_tf.Test_tf(max_vectorize = 100)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 10000
nchunks = 100
Running batched_compute_paired from compute_paired
KS tests calculation completed in 10.654352901037782 seconds.


In [21]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(100, 2), dtype=float64, numpy=
array([[0.012084  , 0.51393678],
       [0.012242  , 0.50299722],
       [0.01244101, 0.4797549 ],
       [0.012232  , 0.49431934],
       [0.01212601, 0.51139932],
       [0.01168601, 0.54716776],
       [0.012153  , 0.49648934],
       [0.01203   , 0.51870604],
       [0.012131  , 0.51312686],
       [0.01215801, 0.50980646],
       [0.012505  , 0.49325656],
       [0.01239   , 0.49321053],
       [0.01354201, 0.4113738 ],
       [0.012035  , 0.51989709],
       [0.01206101, 0.5154787 ],
       [0.01247801, 0.4867854 ],
       [0.01175801, 0.54909493],
       [0.013013  , 0.45425163],
       [0.01145801, 0.57098854],
       [0.01231801, 0.50001895],
       [0.012201  , 0.50498036],
       [0.012091  , 0.52197265],
       [0.012889  , 0.4473419 ],
       [0.012454  , 0.47249988],
       [0.01259501, 0.48018511],
       [0.01224501, 0.50150168],
       [0.01236101, 0.49304492],
       [0.012558  , 0.47561116],
       [0.012252  , 0.5053

In [20]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = False
KSTest_tf.Test_tf(max_vectorize = 1e4)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 10000


Running compute_paired
nchunks = 1
Iterating from 0 to 100 out of 100 .
dist_1_k.shape =  [10000 10000]
dist_2_k.shape =  [10000 10000]
dist_1_k[0,1] =  0.86769471305739987
dist_2_k[0,1] =  -2.3506335124278706
[[0.013040001690387725 0.444238423705101]
 [0.012676003314554691 0.470852010846138]
 [0.012268007025122643 0.49733800947666168]
 ...
 [0.01237300455570221 0.48232251167297363]
 [0.012271005660295486 0.49883547842502596]
 [0.011937004923820495 0.52437033891677853]]
KS tests calculation completed in 7.567521955992561 seconds.


In [21]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(100, 2), dtype=float64, numpy=
array([[0.01304   , 0.44423842],
       [0.012676  , 0.47085201],
       [0.01226801, 0.49733801],
       [0.011757  , 0.54144562],
       [0.01254801, 0.47896242],
       [0.012244  , 0.50820295],
       [0.01160401, 0.54306529],
       [0.012102  , 0.51525854],
       [0.01237   , 0.49209405],
       [0.01224401, 0.50228734],
       [0.01265901, 0.47597909],
       [0.011659  , 0.54803128],
       [0.01255601, 0.47753632],
       [0.011721  , 0.54396492],
       [0.01272001, 0.45630323],
       [0.01221701, 0.50346711],
       [0.012504  , 0.49746184],
       [0.01177001, 0.53685622],
       [0.01205201, 0.52208072],
       [0.01204   , 0.52355376],
       [0.01202801, 0.51523722],
       [0.011915  , 0.52514187],
       [0.012874  , 0.47173555],
       [0.01251901, 0.48415712],
       [0.012034  , 0.52094074],
       [0.012083  , 0.51741341],
       [0.01206901, 0.52432387],
       [0.012368  , 0.48420846],
       [0.01217701, 0.5030

In [22]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = False
KSTest_tf.Test_tf(max_vectorize = 100)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 10000


Running compute_paired
nchunks = 100
Iterating from 0 to 1 out of 100 .
dist_1_k.shape =  [10000 100]
dist_2_k.shape =  [10000 100]
dist_1_k[0,1] =  0.86769471305739987
dist_2_k[0,1] =  -2.3506335124278706
[[0.012308006994426251 0.5040152204036713]]
Iterating from 1 to 2 out of 100 .
dist_1_k.shape =  [10000 100]
dist_2_k.shape =  [10000 100]
dist_1_k[0,1] =  -1.1176023770170542
dist_2_k[0,1] =  0.14133619393519969
[[0.012156001254916192 0.52766206026077267]]
Iterating from 2 to 3 out of 100 .
dist_1_k.shape =  [10000 100]
dist_2_k.shape =  [10000 100]
dist_1_k[0,1] =  -2.4629254905102331
dist_2_k[0,1] =  -0.54861621142110428
[[0.012119006142020225 0.50534133553504945]]
Iterating from 3 to 4 out of 100 .
dist_1_k.shape =  [10000 100]
dist_2_k.shape =  [10000 100]
dist_1_k[0,1] =  -0.38158671985331966
dist_2_k[0,1] =  0.73936311436611091
[[0.012592005282640458 0.46610270857810976]]
Iterating from 4 to 5 out of 100 .
dist_1_k.shape =  [10000 100]
dist_2_k.shape =  [10000 100]
dist_1_k[0,

In [24]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(100, 2), dtype=float64, numpy=
array([[0.01230801, 0.50401522],
       [0.012156  , 0.52766206],
       [0.01211901, 0.50534134],
       [0.01259201, 0.46610271],
       [0.01210201, 0.50048245],
       [0.012333  , 0.48922993],
       [0.011878  , 0.5262731 ],
       [0.012662  , 0.46277149],
       [0.01232201, 0.49563667],
       [0.01290101, 0.43807783],
       [0.01238401, 0.49488417],
       [0.01158001, 0.55229751],
       [0.01310301, 0.44331593],
       [0.012567  , 0.48108905],
       [0.01203701, 0.51714236],
       [0.01260001, 0.46193922],
       [0.011739  , 0.53443239],
       [0.01189201, 0.51883559],
       [0.012029  , 0.51998569],
       [0.01237701, 0.48645956],
       [0.01186701, 0.53927806],
       [0.01308701, 0.43996987],
       [0.01246201, 0.49061798],
       [0.011475  , 0.55933094],
       [0.012505  , 0.47880958],
       [0.011886  , 0.53349633],
       [0.01182801, 0.53102675],
       [0.01204401, 0.51258334],
       [0.01215   , 0.5082

In [42]:
KSTest_tf.Results.results_df

,timestamp,test_name,result_value,is_symb_1,is_symb_2,ndims,niter,batch_size,dtype,small_sample,method,niter_used,batch_size_used,computing_time,backend
0,2023-07-25T12:04:00.195997,KS Test_tf,"((tf.Tensor(0.012365007176995277, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,False,paired,200,10000,13.070821,tensorflow
0,2023-07-25T12:03:20.150864,KS Test_tf,"((tf.Tensor(0.012365007176995277, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,True,paired,200,10000,13.144718,tensorflow
0,2023-07-25T12:02:45.570303,KS Test_tf,"((tf.Tensor(0.012506006583571434, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,True,paired,200,10000,14.955072,tensorflow
0,2023-07-25T12:04:19.579101,KS Test_tf,"((tf.Tensor(0.011603004857897758, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,False,paired,200,10000,19.216098,tensorflow
0,2023-07-25T12:03:46.995924,KS Test_tf,"((tf.Tensor(0.011603004857897758, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,True,paired,200,10000,20.30392,tensorflow


### Run test with TensorFlow (large sample with numeric distribution)

In [27]:
dist_1 = tf.cast(dist.sample(2e6),tf.float64)
dist_2 = tf.cast(dist.sample(2e6),tf.float64)

In [28]:
TwoSampleTestInputs_tf = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                      dist_2_input = dist_2,
                                                      niter = 200,
                                                      batch_size = 10000,
                                                      dtype_input = tf.float64,
                                                      seed_input = 0,
                                                      use_tf = True,
                                                      verbose = True)
print("nsamples",TwoSampleTestInputs_tf.nsamples)
print("batch_size",TwoSampleTestInputs_tf.batch_size)
print("niter",TwoSampleTestInputs_tf.niter)
print("niter * batch_size",TwoSampleTestInputs_tf.niter*TwoSampleTestInputs_tf.batch_size)
print("small_sample",TwoSampleTestInputs_tf.small_sample)

Parsing input distribution...
Input distribution is a numeric numpy array or tf.Tensor.
Parsing input distribution...
Input distribution is a numeric numpy array or tf.Tensor.


nsamples 2000000
batch_size 10000
niter 200
niter * batch_size 2000000
small_sample False


In [29]:
KSTest_tf = GMetrics.KSTest(TwoSampleTestInputs_tf,
                            method = "paired",
                            progress_bar = True,
                            verbose = True)

progress_bar is disabled when using tensorflow mode.


In [32]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = True
KSTest_tf.Test_tf(max_vectorize = 2e4)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 200
batch_size = 10000


nchunks = 1
Running compute_small_paired_best
KS tests calculation completed in 13.144717985996976 seconds.


In [34]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(200, 2), dtype=float64, numpy=
array([[0.01236501, 0.50155848],
       [0.012557  , 0.46779944],
       [0.01220201, 0.49535231],
       [0.011436  , 0.56232608],
       [0.012642  , 0.47314949],
       [0.011978  , 0.52382964],
       [0.01157601, 0.56511108],
       [0.01194901, 0.531531  ],
       [0.01145901, 0.57147783],
       [0.012388  , 0.51928144],
       [0.01244601, 0.47422137],
       [0.011799  , 0.53489203],
       [0.01211301, 0.51055909],
       [0.012578  , 0.4739603 ],
       [0.01157101, 0.57399985],
       [0.012461  , 0.49997327],
       [0.01209   , 0.5142881 ],
       [0.012324  , 0.48684191],
       [0.012319  , 0.50848591],
       [0.01261801, 0.46882253],
       [0.01194201, 0.51977027],
       [0.01286501, 0.47424939],
       [0.01209201, 0.49655082],
       [0.012417  , 0.49690031],
       [0.01221301, 0.50564504],
       [0.012456  , 0.46641654],
       [0.011965  , 0.52207331],
       [0.01224801, 0.50010914],
       [0.012105  , 0.5058

In [35]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = True
KSTest_tf.Test_tf(max_vectorize = 1000)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 200
batch_size = 10000


nchunks = 20
Running batched_compute_paired from compute_paired
KS tests calculation completed in 20.303919759986456 seconds.


In [36]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(200, 2), dtype=float64, numpy=
array([[0.011603  , 0.54519177],
       [0.012892  , 0.4515709 ],
       [0.01179501, 0.53568191],
       [0.01200501, 0.51793178],
       [0.01275001, 0.46413765],
       [0.011634  , 0.54838836],
       [0.011995  , 0.53096928],
       [0.01207401, 0.51544479],
       [0.01263101, 0.48212074],
       [0.01205301, 0.51971121],
       [0.011843  , 0.53355091],
       [0.01209301, 0.52454925],
       [0.01292001, 0.45643294],
       [0.012442  , 0.47788214],
       [0.01267001, 0.48396255],
       [0.01161901, 0.55227904],
       [0.012023  , 0.5214369 ],
       [0.011717  , 0.54127063],
       [0.012954  , 0.46460669],
       [0.01246501, 0.49846961],
       [0.012349  , 0.50661649],
       [0.011685  , 0.54015601],
       [0.012097  , 0.51625734],
       [0.0125    , 0.47694053],
       [0.011787  , 0.53594467],
       [0.01179801, 0.53118063],
       [0.012151  , 0.50230911],
       [0.01221301, 0.51476072],
       [0.011964  , 0.5185

In [37]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = False
KSTest_tf.Test_tf(max_vectorize = 2e4)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 200
batch_size = 10000


nchunks = 1
Running compute_small_paired_best
KS tests calculation completed in 13.070821319997776 seconds.


In [38]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(200, 2), dtype=float64, numpy=
array([[0.01236501, 0.50155848],
       [0.012557  , 0.46779944],
       [0.01220201, 0.49535231],
       [0.011436  , 0.56232608],
       [0.012642  , 0.47314949],
       [0.011978  , 0.52382964],
       [0.01157601, 0.56511108],
       [0.01194901, 0.531531  ],
       [0.01145901, 0.57147783],
       [0.012388  , 0.51928144],
       [0.01244601, 0.47422137],
       [0.011799  , 0.53489203],
       [0.01211301, 0.51055909],
       [0.012578  , 0.4739603 ],
       [0.01157101, 0.57399985],
       [0.012461  , 0.49997327],
       [0.01209   , 0.5142881 ],
       [0.012324  , 0.48684191],
       [0.012319  , 0.50848591],
       [0.01261801, 0.46882253],
       [0.01194201, 0.51977027],
       [0.01286501, 0.47424939],
       [0.01209201, 0.49655082],
       [0.012417  , 0.49690031],
       [0.01221301, 0.50564504],
       [0.012456  , 0.46641654],
       [0.011965  , 0.52207331],
       [0.01224801, 0.50010914],
       [0.012105  , 0.5058

In [39]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = False
KSTest_tf.Test_tf(max_vectorize = 1000)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 200
batch_size = 10000


nchunks = 20
Running batched_compute_paired from compute_paired
KS tests calculation completed in 19.216097511001863 seconds.


In [40]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(200, 2), dtype=float64, numpy=
array([[0.011603  , 0.54519177],
       [0.012892  , 0.4515709 ],
       [0.01179501, 0.53568191],
       [0.01200501, 0.51793178],
       [0.01275001, 0.46413765],
       [0.011634  , 0.54838836],
       [0.011995  , 0.53096928],
       [0.01207401, 0.51544479],
       [0.01263101, 0.48212074],
       [0.01205301, 0.51971121],
       [0.011843  , 0.53355091],
       [0.01209301, 0.52454925],
       [0.01292001, 0.45643294],
       [0.012442  , 0.47788214],
       [0.01267001, 0.48396255],
       [0.01161901, 0.55227904],
       [0.012023  , 0.5214369 ],
       [0.011717  , 0.54127063],
       [0.012954  , 0.46460669],
       [0.01246501, 0.49846961],
       [0.012349  , 0.50661649],
       [0.011685  , 0.54015601],
       [0.012097  , 0.51625734],
       [0.0125    , 0.47694053],
       [0.011787  , 0.53594467],
       [0.01179801, 0.53118063],
       [0.012151  , 0.50230911],
       [0.01221301, 0.51476072],
       [0.011964  , 0.5185

In [41]:
KSTest_tf.Results.results_df

,timestamp,test_name,result_value,is_symb_1,is_symb_2,ndims,niter,batch_size,dtype,small_sample,method,niter_used,batch_size_used,computing_time,backend
0,2023-07-25T12:04:00.195997,KS Test_tf,"((tf.Tensor(0.012365007176995277, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,False,paired,200,10000,13.070821,tensorflow
0,2023-07-25T12:03:20.150864,KS Test_tf,"((tf.Tensor(0.012365007176995277, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,True,paired,200,10000,13.144718,tensorflow
0,2023-07-25T12:02:45.570303,KS Test_tf,"((tf.Tensor(0.012506006583571434, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,True,paired,200,10000,14.955072,tensorflow
0,2023-07-25T12:04:19.579101,KS Test_tf,"((tf.Tensor(0.011603004857897758, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,False,paired,200,10000,19.216098,tensorflow
0,2023-07-25T12:03:46.995924,KS Test_tf,"((tf.Tensor(0.011603004857897758, shape=(), dt...",False,False,100,200,10000,<dtype: 'float64'>,True,paired,200,10000,20.30392,tensorflow


### Run test with TensorFlow (large sample with symbolic distribution)

In [10]:
dist_1 = dist
dist_2 = dist

In [11]:
TwoSampleTestInputs_tf = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                      dist_2_input = dist_2,
                                                      niter = 1000,
                                                      batch_size = 100000,
                                                      dtype_input = tf.float64,
                                                      seed_input = 0,
                                                      use_tf = True,
                                                      verbose = True)
print("nsamples",TwoSampleTestInputs_tf.nsamples)
print("batch_size",TwoSampleTestInputs_tf.batch_size)
print("niter",TwoSampleTestInputs_tf.niter)
print("niter * batch_size",TwoSampleTestInputs_tf.niter*TwoSampleTestInputs_tf.batch_size)
print("small_sample",TwoSampleTestInputs_tf.small_sample)

Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
nsamples 100000000
batch_size 100000
niter 1000
niter * batch_size 100000000
small_sample False


In [12]:
KSTest_tf = GMetrics.KSTest(TwoSampleTestInputs_tf,
                            method = "paired",
                            progress_bar = True,
                            verbose = True)

progress_bar is disabled when using tensorflow mode.


In [13]:
KSTest_tf.method = "paired"
KSTest_tf.Test_tf(max_vectorize = 4e3)

Starting KS tests calculation...


Running KS test in the 'paired' mode.
niter = 1000
batch_size = 100000
Running compute_paired
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 25
Iterating from 0 to 40 out of 1000 .
[[0.0037863055616617204 0.50831023156642918]
 [0.0040195013582706449 0.46302959620952605]
 [0.0038762034475803373 0.49957610666751862]
 ...
 [0.0037456046417355536 0.52442469656467439]
 [0.0038600027188658713 0.50591902077198025]
 [0.00395350344479084 0.48972382485866545]]
Iterating from 40 to 80 out of 1000 .
[[0.0038514088839292525 0.49730458021163942]
 [0.0038227050751447676 0.51675477266311642]
 [0.0039253033697605137 0.4924776178598404]
 ...
 [0.0038066017627716065 0.51250071406364439]
 [0.0036365026980638504 0.5654846078157425]
 [0.0037938044220209122 0.51506197392940523]]
Iterating from 80 to 120 out of 1000 .
[[0.0040215047448873519 0.47079679548740389]
 [0.0037

In [14]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(1000, 2), dtype=float64, numpy=
array([[0.00378631, 0.50831023],
       [0.0040195 , 0.4630296 ],
       [0.0038762 , 0.49957611],
       ...,
       [0.00381521, 0.51448376],
       [0.00387121, 0.50512472],
       [0.0037211 , 0.5293261 ]])>

In [ ]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = True
KSTest_tf.Test_tf(max_vectorize = 1000)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 200
batch_size = 10000


nchunks = 20
Running batched_compute_paired from compute_paired
KS tests calculation completed in 20.303919759986456 seconds.


In [ ]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(200, 2), dtype=float64, numpy=
array([[0.011603  , 0.54519177],
       [0.012892  , 0.4515709 ],
       [0.01179501, 0.53568191],
       [0.01200501, 0.51793178],
       [0.01275001, 0.46413765],
       [0.011634  , 0.54838836],
       [0.011995  , 0.53096928],
       [0.01207401, 0.51544479],
       [0.01263101, 0.48212074],
       [0.01205301, 0.51971121],
       [0.011843  , 0.53355091],
       [0.01209301, 0.52454925],
       [0.01292001, 0.45643294],
       [0.012442  , 0.47788214],
       [0.01267001, 0.48396255],
       [0.01161901, 0.55227904],
       [0.012023  , 0.5214369 ],
       [0.011717  , 0.54127063],
       [0.012954  , 0.46460669],
       [0.01246501, 0.49846961],
       [0.012349  , 0.50661649],
       [0.011685  , 0.54015601],
       [0.012097  , 0.51625734],
       [0.0125    , 0.47694053],
       [0.011787  , 0.53594467],
       [0.01179801, 0.53118063],
       [0.012151  , 0.50230911],
       [0.01221301, 0.51476072],
       [0.011964  , 0.5185

In [ ]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = False
KSTest_tf.Test_tf(max_vectorize = 2e4)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 200
batch_size = 10000


nchunks = 1
Running compute_small_paired_best
KS tests calculation completed in 13.070821319997776 seconds.


In [ ]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(200, 2), dtype=float64, numpy=
array([[0.01236501, 0.50155848],
       [0.012557  , 0.46779944],
       [0.01220201, 0.49535231],
       [0.011436  , 0.56232608],
       [0.012642  , 0.47314949],
       [0.011978  , 0.52382964],
       [0.01157601, 0.56511108],
       [0.01194901, 0.531531  ],
       [0.01145901, 0.57147783],
       [0.012388  , 0.51928144],
       [0.01244601, 0.47422137],
       [0.011799  , 0.53489203],
       [0.01211301, 0.51055909],
       [0.012578  , 0.4739603 ],
       [0.01157101, 0.57399985],
       [0.012461  , 0.49997327],
       [0.01209   , 0.5142881 ],
       [0.012324  , 0.48684191],
       [0.012319  , 0.50848591],
       [0.01261801, 0.46882253],
       [0.01194201, 0.51977027],
       [0.01286501, 0.47424939],
       [0.01209201, 0.49655082],
       [0.012417  , 0.49690031],
       [0.01221301, 0.50564504],
       [0.012456  , 0.46641654],
       [0.011965  , 0.52207331],
       [0.01224801, 0.50010914],
       [0.012105  , 0.5058

In [ ]:
KSTest_tf.method = "paired"
KSTest_tf.Inputs.small_sample = False
KSTest_tf.Test_tf(max_vectorize = 1000)

Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 200
batch_size = 10000


nchunks = 20
Running batched_compute_paired from compute_paired
KS tests calculation completed in 19.216097511001863 seconds.


In [ ]:
KSTest_tf.Results[-1].result_value

<tf.Tensor: shape=(100, 2), dtype=float64, numpy=
array([[0.012472  , 0.48917744],
       [0.012327  , 0.51243107],
       [0.012051  , 0.52256056],
       [0.012031  , 0.51305777],
       [0.01222201, 0.50271434],
       [0.011811  , 0.54559447],
       [0.01260601, 0.489891  ],
       [0.01179601, 0.53170304],
       [0.012299  , 0.49265127],
       [0.01256701, 0.48989817],
       [0.01211501, 0.51079777],
       [0.012547  , 0.47034978],
       [0.01272701, 0.45765291],
       [0.012498  , 0.48412489],
       [0.01235301, 0.48538795],
       [0.01249601, 0.50062007],
       [0.01243201, 0.49086984],
       [0.01259301, 0.46060305],
       [0.01228601, 0.50615565],
       [0.01204501, 0.51633026],
       [0.01306   , 0.44852038],
       [0.011853  , 0.54413055],
       [0.01284501, 0.46872367],
       [0.012149  , 0.52244665],
       [0.012238  , 0.49824879],
       [0.012413  , 0.47987746],
       [0.011753  , 0.55523619],
       [0.012231  , 0.50041708],
       [0.01240101, 0.4944

In [ ]:
KSTest_tf.Results.results_df

,timestamp,test_name,result_value,is_symb_1,is_symb_2,ndims,niter,batch_size,dtype,small_sample,method,niter_used,batch_size_used,computing_time,backend
0,2023-07-25T08:10:03.990370,KS Test_tf,"((tf.Tensor(0.012363004162907601, shape=(), dt...",False,False,100,100,10000,<dtype: 'float64'>,False,paired,100,10000,7.73814,tensorflow
0,2023-07-25T08:11:20.838747,KS Test_tf,"((tf.Tensor(0.012363004162907601, shape=(), dt...",False,False,100,100,10000,<dtype: 'float64'>,True,paired,100,10000,7.80006,tensorflow
0,2023-07-25T08:09:51.835187,KS Test_tf,"((tf.Tensor(0.012363004162907601, shape=(), dt...",False,False,100,100,10000,<dtype: 'float64'>,True,paired,100,10000,9.297316,tensorflow


### Run test with Numpy 

In [38]:
dist_1_np = dist_1.numpy()
dist_2_np = dist_2.numpy()

In [42]:
TwoSampleTestInputs_np = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1_np,
                                                      dist_2_input = dist_2_np,
                                                      niter = 1000,
                                                      batch_size = 10000, 
                                                      dtype_input = np.float64,
                                                      seed_input = 0,
                                                      use_tf = False,
                                                      verbose = True)
print("nsamples",TwoSampleTestInputs_np.nsamples)
print("batch_size",TwoSampleTestInputs_np.batch_size)
print("niter",TwoSampleTestInputs_np.niter)
print("niter * batch_size",TwoSampleTestInputs_np.niter*TwoSampleTestInputs_np.batch_size)

Parsing input distribution...
Input distribution is a numberic numpy array or tf.Tensor
Parsing input distribution...
Input distribution is a numberic numpy array or tf.Tensor
nsamples 1000000
batch_size 10000
niter 1000
niter * batch_size 10000000


In [43]:
KSTest_np = GMetrics.KSTest(TwoSampleTestInputs_np,
                            method = "paired",
                            progress_bar = True,
                            verbose = True)

In [44]:
KSTest_np.method = "paired"
KSTest_np.Inputs.small_sample = True
KSTest_np.Test_np()

Starting two-sample test calculation...
Running KS test in the 'small sample' and 'paired' mode.
niter = 1000
batch_size = 1000


Iterations:   0%|          | 0/1000 [00:00<?, ?it/s]

Iterations:   5%|▌         | 50/1000 [00:00<00:01, 495.22it/s]

Running KS test in the 'small sample' and 'paired' mode.


Iterations: 100%|██████████| 1000/1000 [00:00<00:00, 1013.50it/s]

Two-sample test calculation completed in 0.9894446188118309 seconds.


In [45]:
KSTest_np.method = "paired"
KSTest_np.Inputs.small_sample = False
KSTest_np.Test_np()

Starting two-sample test calculation...
Running KS test in the 'large sample' and 'paired' mode.
niter = 1000
batch_size = 1000


Iterations:  11%|█         | 109/1000 [00:00<00:00, 1084.65it/s]

Running KS test in the 'large sample' and 'paired' mode.


Iterations: 100%|██████████| 1000/1000 [00:00<00:00, 1100.74it/s]

Two-sample test calculation completed in 0.9108401911798865 seconds.


In [47]:
KSTest_np.method = "crossed"
KSTest_np.Inputs.small_sample = True
KSTest_np.Test_np()

Starting two-sample test calculation...
Running KS test in the 'small sample' and 'unpaired' mode.
niter = 961
batch_size = 10000


Iterations:   1%|          | 9/961 [00:00<00:11, 81.30it/s]

Running KS test in the 'small sample' and 'crossed' mode.


Iterations: 100%|██████████| 961/961 [00:08<00:00, 119.62it/s]


Two-sample test calculation completed in 8.036634234944358 seconds.


In [48]:
KSTest_np.method = "crossed"
KSTest_np.Inputs.small_sample = False
KSTest_np.Test_np()

Starting two-sample test calculation...
Running KS test in the 'large sample' and 'unpaired' mode.
niter = 961
batch_size = 10000


Iterations:   0%|          | 0/961 [00:00<?, ?it/s]

Iterations:   1%|          | 12/961 [00:00<00:08, 116.19it/s]

Running KS test in the 'large sample' and 'crossed' mode.


Iterations: 100%|██████████| 961/961 [00:08<00:00, 117.89it/s]

Two-sample test calculation completed in 8.154056519968435 seconds.


In [49]:
KSTest_np.Results.results_df

,timestamp,test_name,result_value,is_symb_1,is_symb_2,ndims,niter,batch_size,dtype,small_sample,method,niter_used,batch_size_used,computing_time,backend
0,2023-07-12T16:38:35.835787,KS Test_np,"[[0.04, 0.4715141297736493], [0.034, 0.6126645...",False,False,2,1000,10000,float64,False,paired,1000,1000,0.91084,numpy
0,2023-07-12T16:38:34.321675,KS Test_np,"[[0.04, 0.4715141297736493], [0.034, 0.6126645...",False,False,2,1000,10000,float64,True,paired,1000,1000,0.989445,numpy
0,2023-07-12T16:38:58.206853,KS Test_np,"[[0.015, 0.2117107566537918], [0.0129, 0.38862...",False,False,2,1000,10000,float64,True,crossed,961,10000,8.036634,numpy
0,2023-07-12T16:39:06.413289,KS Test_np,"[[0.015, 0.2117107566537918], [0.0129, 0.38862...",False,False,2,1000,10000,float64,False,crossed,961,10000,8.154057,numpy


### Run test scan (1D, "paired")

In [2]:
ncomp = 3
ndims = 1
seed = 0
loc = np.array(np.zeros(ndims,dtype=np.float64),dtype=np.float64)
scale = np.array(np.ones(ndims,dtype=np.float64),dtype=np.float64)
dist = tfp.distributions.Normal(loc=loc, scale=scale)
#dist = MixtureDistributions.MixtureGaussian(ncomp,ndims,seed)
print(dist.dtype)

<dtype: 'float64'>


In [3]:
dist_1 = dist
dist_2 = dist

In [4]:
TwoSampleTestInputs_tf = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                      dist_2_input = dist_2,
                                                      niter = 1000,
                                                      batch_size = 10000, 
                                                      dtype_input = tf.float64,
                                                      seed_input = 0,
                                                      use_tf = True,
                                                      verbose = True)
print("nsamples",TwoSampleTestInputs_tf.nsamples)
print("batch_size",TwoSampleTestInputs_tf.batch_size)
print("niter",TwoSampleTestInputs_tf.niter)
print("niter * batch_size",TwoSampleTestInputs_tf.niter*TwoSampleTestInputs_tf.batch_size)

Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.


nsamples 10000000
batch_size 10000
niter 1000
niter * batch_size 10000000


In [7]:
KSTest_tf = GMetrics.KSTest(TwoSampleTestInputs_tf,
                            method = "paired",
                            progress_bar = False,
                            verbose = True)

In [8]:
for batch_size in [1000,10000,100000]:
    for niter in [100,1000,10000]:
            print("\n========================================")
            KSTest_tf.Inputs.batch_size = batch_size
            KSTest_tf.Inputs.niter = niter
            print("niter:", niter)
            print("batch_size:", batch_size)
            print("----------------------------------------")
            print("Running TF test...")
            KSTest_tf.Test_tf(max_vectorize = int(5e7 // batch_size))
            print("----------------------------------------")
            print("Running NP test...")
            KSTest_tf.Test_np()


niter: 100
batch_size: 1000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 1000


Running compute_paired
KS tests calculation completed in 1.3419655699981377 seconds.
----------------------------------------
Running NP test...
Starting two-sample test calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 1000
Running numpy paired
Two-sample test calculation completed in 0.08238235098542646 seconds.

niter: 1000
batch_size: 1000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 1000
batch_size = 1000
Running compute_paired
KS tests calculation completed in 1.7657665300066583 seconds.
----------------------------------------
Running NP test...
Starting two-sample test calculation...
Running KS test in the 'paired' mode.
niter = 1000
batch_size = 1000
Running numpy paired
Two-sample test calculation completed in 0.48566309799207374 seconds.

niter: 10000
batch_size: 1000
----------------------------------------
Running TF test...
Starting KS tests calculatio

In [9]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
df = pd.DataFrame(KSTest_tf.Results.results_df)
display(df)

,timestamp,test_name,result_value,is_symb_1,is_symb_2,ndims,niter,batch_size,dtype,small_sample,method,niter_used,batch_size_used,computing_time,backend
0,2023-07-25T16:32:41.151233,KS Test_np,"[[0.024, 0.9357699014782725], [0.026, 0.888173...",True,True,1,100,1000,<dtype: 'float64'>,True,paired,100,1000,0.082382,numpy
0,2023-07-25T16:32:56.593041,KS Test_np,"[[0.0105, 0.6398931694241892], [0.0087, 0.8435...",True,True,1,100,10000,<dtype: 'float64'>,True,paired,100,10000,0.40887,numpy
0,2023-07-25T16:32:43.662406,KS Test_np,"[[0.024, 0.9357699014782725], [0.026, 0.888173...",True,True,1,1000,1000,<dtype: 'float64'>,True,paired,1000,1000,0.485663,numpy
0,2023-07-25T16:32:41.041880,KS Test_tf,"((tf.Tensor(0.03799998760223389, shape=(), dty...",True,True,1,100,1000,<dtype: 'float64'>,True,paired,100,1000,1.341966,tensorflow
0,2023-07-25T16:32:56.165991,KS Test_tf,"((tf.Tensor(0.01130002737045288, shape=(), dty...",True,True,1,100,10000,<dtype: 'float64'>,True,paired,100,10000,1.349851,tensorflow
0,2023-07-25T16:33:52.841597,KS Test_tf,"((tf.Tensor(0.0025099515914916992, shape=(), d...",True,True,1,100,100000,<dtype: 'float64'>,True,paired,100,100000,1.52655,tensorflow
0,2023-07-25T16:32:43.139928,KS Test_tf,"((tf.Tensor(0.030000001192092896, shape=(), dt...",True,True,1,1000,1000,<dtype: 'float64'>,True,paired,1000,1000,1.765767,tensorflow
0,2023-07-25T16:32:58.860792,KS Test_tf,"((tf.Tensor(0.008900046348571777, shape=(), dt...",True,True,1,1000,10000,<dtype: 'float64'>,True,paired,1000,10000,1.975167,tensorflow
0,2023-07-25T16:33:58.768473,KS Test_tf,"((tf.Tensor(0.0021400004625320435, shape=(), d...",True,True,1,1000,100000,<dtype: 'float64'>,True,paired,1000,100000,2.582919,tensorflow
0,2023-07-25T16:33:55.911707,KS Test_np,"[[0.0045800000000000285, 0.24429456497117985],...",True,True,1,100,100000,<dtype: 'float64'>,True,paired,100,100000,2.996039,numpy


### Run test scan (10D, "paired")

In [10]:
ncomp = 3
ndims = 10
seed = 0
loc = np.array(np.zeros(ndims,dtype=np.float64),dtype=np.float64)
scale = np.array(np.ones(ndims,dtype=np.float64),dtype=np.float64)
dist = tfp.distributions.Normal(loc=loc, scale=scale)
#dist = MixtureDistributions.MixtureGaussian(ncomp,ndims,seed)
print(dist.dtype)

<dtype: 'float64'>


In [11]:
dist_1 = dist
dist_2 = dist

In [12]:
TwoSampleTestInputs_tf = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                      dist_2_input = dist_2,
                                                      niter = 1000,
                                                      batch_size = 10000, 
                                                      dtype_input = tf.float64,
                                                      seed_input = 0,
                                                      use_tf = True,
                                                      verbose = True)
print("nsamples",TwoSampleTestInputs_tf.nsamples)
print("batch_size",TwoSampleTestInputs_tf.batch_size)
print("niter",TwoSampleTestInputs_tf.niter)
print("niter * batch_size",TwoSampleTestInputs_tf.niter*TwoSampleTestInputs_tf.batch_size)

Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
nsamples 10000000
batch_size 10000
niter 1000
niter * batch_size 10000000


In [13]:
KSTest_tf = GMetrics.KSTest(TwoSampleTestInputs_tf,
                            method = "paired",
                            progress_bar = False,
                            verbose = True)

In [14]:
for batch_size in [1000,10000,100000]:
    for niter in [100,1000]:
            print("\n========================================")
            KSTest_tf.Inputs.batch_size = batch_size
            KSTest_tf.Inputs.niter = niter
            print("niter:", niter)
            print("batch_size:", batch_size)
            print("----------------------------------------")
            print("Running TF test...")
            KSTest_tf.Test_tf(max_vectorize = int(5e7 // batch_size))
            print("----------------------------------------")
            print("Running NP test...")
            KSTest_tf.Test_np()


niter: 100
batch_size: 1000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 1000
Running compute_paired
KS tests calculation completed in 2.208421384973917 seconds.
----------------------------------------
Running NP test...
Starting two-sample test calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 1000
Running numpy paired
Two-sample test calculation completed in 0.45784362201811746 seconds.

niter: 1000
batch_size: 1000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 1000
batch_size = 1000
Running compute_paired
KS tests calculation completed in 4.884976493020076 seconds.
----------------------------------------
Running NP test...
Starting two-sample test calculation...
Running KS test in the 'paired' mode.
niter = 1000
batch_size = 1000
Running numpy paired
T

In [16]:
for batch_size in [1000,10000,100000]:
    for niter in [10000]:
        print("\n========================================")
        KSTest_tf.Inputs.batch_size = batch_size
        KSTest_tf.Inputs.niter = niter
        print("niter:", niter)
        print("batch_size:", batch_size)
        print("----------------------------------------")
        print("Running TF test...")
        KSTest_tf.Test_tf(max_vectorize = int(5e7 // batch_size))


niter: 10000
batch_size: 1000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 10000
batch_size = 1000
Running compute_paired
nchunks = 2
Iterating from 0 to 5000 out of 10000 .
Iterating from 5000 to 10000 out of 10000 .
KS tests calculation completed in 28.610340971965343 seconds.

niter: 10000
batch_size: 10000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 10000
batch_size = 10000
Running compute_paired
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 20
Iterating from 0 to 500 out of 10000 .
Iterating from 500 to 1000 out of 10000 .
Iterating from 1000 to 1500 out of 10000 .
Iterating from 1500 to 2000 out of 10000 .
Iterating from 2000 to 2500 out of 10000 .
Itera

In [17]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
df = pd.DataFrame(KSTest_tf.Results.results_df)
display(df)

,timestamp,test_name,result_value,is_symb_1,is_symb_2,ndims,niter,batch_size,dtype,small_sample,method,niter_used,batch_size_used,computing_time,backend
0,2023-07-25T16:45:15.667279,KS Test_np,"[[0.045099999999999994, 0.358428950637731], [0...",True,True,10,100,1000,<dtype: 'float64'>,True,paired,100,1000,0.457844,numpy
0,2023-07-25T16:45:27.657051,KS Test_tf,"((tf.Tensor(0.012250006198883057, shape=(), dt...",True,True,10,100,10000,<dtype: 'float64'>,True,paired,100,10000,1.963437,tensorflow
0,2023-07-25T16:45:15.182061,KS Test_tf,"((tf.Tensor(0.03570000231266022, shape=(), dty...",True,True,10,100,1000,<dtype: 'float64'>,True,paired,100,1000,2.208421,tensorflow
0,2023-07-25T16:46:24.856467,KS Test_tf,"((tf.Tensor(0.004137992113828659, shape=(), dt...",True,True,10,100,100000,<dtype: 'float64'>,True,paired,100,100000,2.505057,tensorflow
0,2023-07-25T16:45:32.036060,KS Test_np,"[[0.01341, 0.47881218574956075], [0.0125, 0.45...",True,True,10,100,10000,<dtype: 'float64'>,True,paired,100,10000,4.309498,numpy
0,2023-07-25T16:45:25.453299,KS Test_np,"[[0.045099999999999994, 0.358428950637731], [0...",True,True,10,1000,1000,<dtype: 'float64'>,True,paired,1000,1000,4.618264,numpy
0,2023-07-25T16:45:20.760987,KS Test_tf,"((tf.Tensor(0.03760000169277191, shape=(), dty...",True,True,10,1000,1000,<dtype: 'float64'>,True,paired,1000,1000,4.884976,tensorflow
0,2023-07-25T16:45:39.201319,KS Test_tf,"((tf.Tensor(0.013390000164508819, shape=(), dt...",True,True,10,1000,10000,<dtype: 'float64'>,True,paired,1000,10000,6.838129,tensorflow
0,2023-07-25T16:47:11.326426,KS Test_tf,"((tf.Tensor(0.003839007019996643, shape=(), dt...",True,True,10,1000,100000,<dtype: 'float64'>,False,paired,1000,100000,14.866724,tensorflow
0,2023-07-25T16:55:20.217721,KS Test_tf,"((tf.Tensor(0.03580001592636108, shape=(), dty...",True,True,10,10000,1000,<dtype: 'float64'>,True,paired,10000,1000,28.610341,tensorflow


### Run test scan (100D, "paired")

In [2]:
ncomp = 3
ndims = 100
seed = 0
loc = np.array(np.zeros(ndims,dtype=np.float64),dtype=np.float64)
scale = np.array(np.ones(ndims,dtype=np.float64),dtype=np.float64)
dist = tfp.distributions.Normal(loc=loc, scale=scale)
#dist = MixtureDistributions.MixtureGaussian(ncomp,ndims,seed)
print(dist.dtype)

<dtype: 'float64'>


In [3]:
dist_1 = dist
dist_2 = dist

In [4]:
TwoSampleTestInputs_tf = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                      dist_2_input = dist_2,
                                                      niter = 1000,
                                                      batch_size = 10000, 
                                                      dtype_input = tf.float64,
                                                      seed_input = 0,
                                                      use_tf = True,
                                                      verbose = True)
print("nsamples",TwoSampleTestInputs_tf.nsamples)
print("batch_size",TwoSampleTestInputs_tf.batch_size)
print("niter",TwoSampleTestInputs_tf.niter)
print("niter * batch_size",TwoSampleTestInputs_tf.niter*TwoSampleTestInputs_tf.batch_size)

Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
nsamples 10000000
batch_size 10000
niter 1000
niter * batch_size 10000000


In [5]:
KSTest_tf = GMetrics.KSTest(TwoSampleTestInputs_tf,
                            method = "paired",
                            progress_bar = False,
                            verbose = True)

In [6]:
for batch_size in [1000,10000,100000]:
    for niter in [100,1000]:
            print("\n========================================")
            KSTest_tf.Inputs.batch_size = batch_size
            KSTest_tf.Inputs.niter = niter
            print("niter:", niter)
            print("batch_size:", batch_size)
            print("----------------------------------------")
            print("Running TF test...")
            KSTest_tf.Test_tf(max_vectorize = int(5e7 // batch_size))
            print("----------------------------------------")
            print("Running NP test...")
            KSTest_tf.Test_np()


niter: 100
batch_size: 1000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 1000
Running compute_paired
KS tests calculation completed in 6.553407329018228 seconds.
----------------------------------------
Running NP test...
Starting two-sample test calculation...
Running KS test in the 'paired' mode.
niter = 100
batch_size = 1000
Running numpy paired
Two-sample test calculation completed in 4.530397376976907 seconds.

niter: 1000
batch_size: 1000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 1000
batch_size = 1000
Running compute_paired
nchunks = 2
Iterating from 0 to 500 out of 1000 .
Iterating from 500 to 1000 out of 1000 .
KS tests calculation completed in 29.8214386330219 seconds.
----------------------------------------
Running NP test...
Starting two-sample test calculation...
Run

In [ ]:
for batch_size in [1000,10000,100000]:
    for niter in [10000]:
        print("\n========================================")
        KSTest_tf.Inputs.batch_size = batch_size
        KSTest_tf.Inputs.niter = niter
        print("niter:", niter)
        print("batch_size:", batch_size)
        print("----------------------------------------")
        print("Running TF test...")
        KSTest_tf.Test_tf(max_vectorize = int(5e7 // batch_size))


niter: 10000
batch_size: 1000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 10000
batch_size = 1000
Running compute_paired
nchunks = 2
Iterating from 0 to 5000 out of 10000 .
Iterating from 5000 to 10000 out of 10000 .
KS tests calculation completed in 28.610340971965343 seconds.

niter: 10000
batch_size: 10000
----------------------------------------
Running TF test...
Starting KS tests calculation...
Running KS test in the 'paired' mode.
niter = 10000
batch_size = 10000
Running compute_paired
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 20
Iterating from 0 to 500 out of 10000 .
Iterating from 500 to 1000 out of 10000 .
Iterating from 1000 to 1500 out of 10000 .
Iterating from 1500 to 2000 out of 10000 .
Iterating from 2000 to 2500 out of 10000 .
Itera

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
df = pd.DataFrame(KSTest_tf.Results.results_df)
display(df)

,timestamp,test_name,result_value,is_symb_1,is_symb_2,ndims,niter,batch_size,dtype,small_sample,method,niter_used,batch_size_used,computing_time,backend
0,2023-07-25T16:45:15.667279,KS Test_np,"[[0.045099999999999994, 0.358428950637731], [0...",True,True,10,100,1000,<dtype: 'float64'>,True,paired,100,1000,0.457844,numpy
0,2023-07-25T16:45:27.657051,KS Test_tf,"((tf.Tensor(0.012250006198883057, shape=(), dt...",True,True,10,100,10000,<dtype: 'float64'>,True,paired,100,10000,1.963437,tensorflow
0,2023-07-25T16:45:15.182061,KS Test_tf,"((tf.Tensor(0.03570000231266022, shape=(), dty...",True,True,10,100,1000,<dtype: 'float64'>,True,paired,100,1000,2.208421,tensorflow
0,2023-07-25T16:46:24.856467,KS Test_tf,"((tf.Tensor(0.004137992113828659, shape=(), dt...",True,True,10,100,100000,<dtype: 'float64'>,True,paired,100,100000,2.505057,tensorflow
0,2023-07-25T16:45:32.036060,KS Test_np,"[[0.01341, 0.47881218574956075], [0.0125, 0.45...",True,True,10,100,10000,<dtype: 'float64'>,True,paired,100,10000,4.309498,numpy
0,2023-07-25T16:45:25.453299,KS Test_np,"[[0.045099999999999994, 0.358428950637731], [0...",True,True,10,1000,1000,<dtype: 'float64'>,True,paired,1000,1000,4.618264,numpy
0,2023-07-25T16:45:20.760987,KS Test_tf,"((tf.Tensor(0.03760000169277191, shape=(), dty...",True,True,10,1000,1000,<dtype: 'float64'>,True,paired,1000,1000,4.884976,tensorflow
0,2023-07-25T16:45:39.201319,KS Test_tf,"((tf.Tensor(0.013390000164508819, shape=(), dt...",True,True,10,1000,10000,<dtype: 'float64'>,True,paired,1000,10000,6.838129,tensorflow
0,2023-07-25T16:47:11.326426,KS Test_tf,"((tf.Tensor(0.003839007019996643, shape=(), dt...",True,True,10,1000,100000,<dtype: 'float64'>,False,paired,1000,100000,14.866724,tensorflow
0,2023-07-25T16:46:56.208673,KS Test_np,"[[0.00409500000000001, 0.49102287946199513], [...",True,True,10,100,100000,<dtype: 'float64'>,True,paired,100,100000,30.822527,numpy
